In [1]:
import urllib
import requests
import json
import datetime
from urllib.parse import quote, urlencode

import numpy as np
import pandas as pd
import pickle

In [2]:
HOST = "https://uk.wikipedia.org/w/api.php"

In [3]:
def getViews(page, start, end, project='uk.wikipedia'):
    """
    Get number of Views for certain page for every day between start and end. 
    params: page: which pages you want data for
    params: start: starting from this date
    params: end: end on this date
    params: project: which wiki
    return: df: created dataframe with timeseries
    
    """
    base_url = "https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/%s/all-access/all-agents/%s/daily/%s/%s" % (project,page,start,end)
    data = urllib.request.urlopen(base_url)
    dataJson = json.loads(data.read().decode('utf-8'))['items']
    df = pd.DataFrame(dataJson) [['views','timestamp']]
    df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y%m%d%H')
    return df

In [4]:
def GetRevisions_andAge(pageTitle, timestamp):
    """
    Get number of Revisions for certain day 
    params: pageTitle: which pages you want data for
    params: timestamp: on which date
    return: users: all user revisions
    return: delta.days: age of page in days (from first revision till timestamp)
    """
    params = {}
    params["action"] = "query"
    params["format"] = "json"
    params["prop"] = "revisions"
    params["rvlimit"] = "1000"
    params["rvstart"] = timestamp #It goes down (new on top)
    users = []
    revs = []
    
    while True:
        response = urllib.request.urlopen(HOST + "?" + urlencode(params) + "&titles=" + quote(pageTitle)).read().decode('utf-8')
        dataJson = json.loads(response)
        pages = dataJson["query"]["pages"]
        key = list(pages.keys())[0]
        revisions = pages[key]['revisions']
        users = users + [rev["user"] for rev in revisions]
        revs = revs + [rev for rev in revisions]
        
        if "continue" in dataJson.keys():
            cont = dataJson["continue"]["rvcontinue"]
            params["rvcontinue"] = cont
        else:
            break
    last_revision = revs[-1]
    page_born = datetime.datetime.strptime(last_revision['timestamp'], '%Y-%m-%dT%H:%M:%SZ')
    timestamp_datetime = datetime.datetime.strptime(timestamp, '%Y-%m-%dT%H:%M:%SZ')

    # today = datetime.datetime.today()
    delta = timestamp_datetime - page_born
    return users, delta.days

In [5]:
def create_timeseries(page_titles, str_date_from, number_of_days):
    """
    Create timeseries start from given date and go older
    params: page_titles: which pages you want data for
    params: str_date_from: this and number_of_days before will be taken
    params: number_of_days: how many days back from start should we look
    return: df: created dataframe with timeseries
    """
    datetime_start = datetime.datetime.strptime(str_date_from, "%Y-%m-%dT%H:%M:%SZ")
    df = pd.DataFrame(columns=['timestamp','page_name','revisions_count','contributors_count','age_of_page_days'])
    for i in range(number_of_days):
        time_query  = datetime_start - datetime.timedelta(days=i)
        time_query_revs_str = time_query.strftime("%Y-%m-%dT%H:%M:%SZ")
        time_query_views_str = time_query.strftime("%Y%m%d00")
        print(time_query_revs_str)
        
        print("Run for {}, when took {} days back".format(time_query_revs_str, i))
        for page_name in page_titles:
            try:
                users_updated, age_of_page_days = GetRevisions_andAge(page_name,time_query_revs_str)
                revisions = len(users_updated)
                contributors = len(set(users_updated))
                try:
                    num_of_views = getViews(page_name,time_query_views_str,time_query_views_str) 
                except:
                    num_of_views = 0
                df = df.append({'timestamp':time_query_revs_str,
                            'page_name':page_name,
                            'revisions_count':revisions,
                            'contributors_count':contributors,
                            'age_of_page_days':age_of_page_days,
                            'num_of_views':num_of_views
                            }, ignore_index=True)
            except  Exception as e:
                print("Error for page {} on {} day before".format(page_name, i))
                print("Error message: {}".format(e))
    return df

In [6]:
TEST_SUBSET = 100
DAYS = 40

In [ ]:
#Load Yurii data

pkl_file = open('pages.pkl', 'rb')
data1 = pickle.load(pkl_file)

titles = [i[0].replace(" ", "_") for i in data1] # only Ukr titles
#titles = titles[-TEST_SUBSET:]
my_df = create_timeseries(titles, "2018-06-07T12:00:00Z", DAYS)
my_df.to_csv('{}days{}pages.csv'.format(DAYS,TEST_SUBSET))

2018-06-07T12:00:00Z
Run for 2018-06-07T12:00:00Z, when took 0 days back


In [ ]:
df = pd.read_csv("C:/Users/kzorina/Studing/MMDS/not_eng_titles_500.csv")
titles = df['uk_title'].tolist()
titles = [i.replace(" ", "_") for i in titles]
№titles = titles[50:150]
not_eng_df = create_timeseries(titles, "2018-06-07T12:00:00Z", DAYS)
not_eng_df.to_csv('Not_translated{}days{}pages.csv'.format(DAYS,100))

### Drafts

In [ ]:
#Demo how function works
users_updated, age_of_page_days = GetRevisions("Львів","2018-01-15T14:56:00Z")
revisions = len(users_updated)
contributors = len(set(users_updated))

print("Revisions: ".format(revisions))
print("contributors".format(contributors))
print("Age of page".format(age_of_page_days))

In [ ]:
# In development....
def get_lang():
    """
    For every title find its translations (among 10 lang for now)
    + get their first revision
    """
    